In [1]:
import pandas as pd
from pathlib import Path
import torch
from transformers import BertTokenizer, BertModel

/home/yutanagano/Projects/sceptr_publication_analyses/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
PROJECT_ROOT = Path.cwd().parent.resolve()

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")

In [4]:
tcr_chains = pd.read_csv(PROJECT_ROOT/"tcr_data"/"preprocessed"/"benchmarking"/"vdjdb_cleaned_tcr_chain_aa_seqs.csv").dropna()

In [5]:
tokeniser = BertTokenizer.from_pretrained("Rostlab/prot_bert")
model = BertModel.from_pretrained("Rostlab/prot_bert")
model = model.to(device)

In [6]:
tcr_chains

,v,cdr3,j,aa
0,TRAV27*01,CAGAGSQGNLIF,TRAJ42*01,MVLKFSVSILWIQLAWVSTQLLEQSPQFLSIQEGENLTVYCNSSSV...
1,TRAV24*01,CARNTGNQFYF,TRAJ49*01,MEKNPLAAPLLILWFHLDCVSSILNVEQSPQSLHVQEGDSTNFTCS...
2,TRAV5*01,CAEDNNARLMF,TRAJ31*01,MKTFAGFSFLFLWLQLDCMSRGEDVEQSLFLSVREGDSSVINCTYT...
3,TRAV24*01,CAFITGNQFYF,TRAJ49*01,MEKNPLAAPLLILWFHLDCVSSILNVEQSPQSLHVQEGDSTNFTCS...
4,TRAV26-2*01,CILDNNNDMRF,TRAJ43*01,MKLVTSITVLLSLGIMGDAKTTQPNSMESNEEEPVHLPCNHSTISG...
...,...,...,...,...
6234,TRBV6-2*01,CASSYRQGQPQHF,TRBJ1-5*01,MSLGLLCCGAFSLLWAGPVNAGVTQTPKFRVLKTGQSMTLLCAQDM...
6235,TRBV6-6*01,CASRNRQQTLYGYTF,TRBJ1-2*01,MSISLLCCAAFPLLWAGPVNAGVTQTPKFRILKIGQSMTLQCTQDM...
6236,TRBV5-5*01,CASSWRQGGSIRESYTF,TRBJ1-2*01,MGPGLLCWVLLCLLGAGPVDAGVTQSPTHLIKTRGQQVTLRCSPIS...
6237,TRBV6-4*01,CASSDRGTGLNGYTF,TRBJ1-2*01,MRIRLLCCVAFSLLWAGPVIAGITQAPTSQILAAGRRMTLRCTQDM...


In [7]:
sequences = tcr_chains.aa.map(lambda seq: " ".join(list(seq))).to_list()

In [9]:
tokenised = tokeniser(sequences[:4], return_tensors="pt", padding=True)
tokenised = {k: v.to(device) for k, v in tokenised.items()}
representations = model(**tokenised).pooler_output

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 7.77 GiB of which 58.00 MiB is free. Including non-PyTorch memory, this process has 7.24 GiB memory in use. Of the allocated memory 6.53 GiB is allocated by PyTorch, and 589.66 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)